In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json

url = "https://fbref.com/en/squads/b8fd03ef/2022-2023/Manchester-City-Stats"

response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Znajdowanie tabeli z zawodnikami
players_table = soup.find('table', {'id': 'stats_standard_9'})

# Lista do przechowywania danych o zawodnikach
players_data = []

# Iterowanie przez wiersze tabeli, aby zebrać dane zawodników
for row in players_table.find('tbody').find_all('tr'):
    player_name = row.find('th', {'data-stat': 'player'}).text.strip()
    player_link = row.find('th', {'data-stat': 'player'}).find('a')
    
    if player_link:
        player_id = player_link['href'].split('/')[3]
        players_data.append({'player_id': player_id, 'player_name': player_name})

# Tworzenie DataFrame z zebranych danych
df = pd.DataFrame(players_data)


In [19]:
players_data

[{'player_id': '3bb7b8b4', 'player_name': 'Ederson'},
 {'player_id': '6434f10d', 'player_name': 'Rodri'},
 {'player_id': '1f44ac21', 'player_name': 'Erling Haaland'},
 {'player_id': 'e46012d4', 'player_name': 'Kevin De Bruyne'},
 {'player_id': '819b3158', 'player_name': 'İlkay Gündoğan'},
 {'player_id': '89ac64a6', 'player_name': 'Manuel Akanji'},
 {'player_id': '3eb22ec9', 'player_name': 'Bernardo Silva'},
 {'player_id': 'b0b4fd3e', 'player_name': 'Jack Grealish'},
 {'player_id': '31c69ef1', 'player_name': 'Rúben Dias'},
 {'player_id': '86dd77d1', 'player_name': 'Kyle Walker'},
 {'player_id': '892d5bb1', 'player_name': 'Riyad Mahrez'},
 {'player_id': 'eaeca114', 'player_name': 'Nathan Aké'},
 {'player_id': 'ed1e53f3', 'player_name': 'Phil Foden'},
 {'player_id': '5eecec3d', 'player_name': 'John Stones'},
 {'player_id': 'bd6351cd', 'player_name': 'João Cancelo'},
 {'player_id': '15ab5a2b', 'player_name': 'Julián Álvarez'},
 {'player_id': '119b9a8e', 'player_name': 'Aymeric Laporte'},
 

In [6]:
file_name= 'player_names_dict.json'
with open(file_name, 'r', encoding='utf-8') as file:
    player_dict = json.load(file)

In [ ]:
player_dict

In [27]:
class Player:
    
    def __init__(self, player_id, player_dict, player_name, position):
        self.player_id = player_id
        self.player_dict = player_dict
        self.player_name = player_name
        self.position= position
    
    def fetch_player_data(self):

        try:
            df_list = pd.read_html(f'https://fbref.com/en/players/{self.player_id}/{self.player_name}')
             # Próba pobrania danych
                
            if position== 'GK':
                index= next(i for i, df in enumerate(df_list) if len(df.columns)==25)
                df_list=df_list[index:index+10]
            else:
                index= next(i for i, df in enumerate(df_list) if len(df.columns)==36)
                df_list=df_list[index:index+9]
                
            keys = list(self.player_dict.keys())

                # Upewnienie się, że mamy odpowiednią liczbę DataFrame'ów
                if len(keys) != len(df_list):
                    raise ValueError("Liczba kluczy i DataFrame'ów nie jest równa.")

                # Tworzenie słownika DataFrame'ów
                dict_dfs = {keys[i]: df for i, df in enumerate(df_list)}

                # Przypisywanie MultiIndex do każdego DataFrame'u
                for key in keys:
                    if key in dict_dfs and len(self.player_dict[key]) == len(dict_dfs[key].columns):
                        player_dict_tup = [tuple(col) for col in self.player_dict[key]]
                        multi_index = pd.MultiIndex.from_tuples(player_dict_tup)
                        dict_dfs[key].columns = multi_index
                    else:
                        raise ValueError(f"Nie można przypisać MultiIndex do DataFrame'u dla klucza '{key}'.")

            return dict_dfs

        except Exception as e:
            # Obsługa wyjątków, np. problemów z połączeniem internetowym, błędów w danych itp.
            print(f"Wystąpił błąd: {e}")
            return None



In [80]:
df_list = pd.read_html(f'https://fbref.com/en/players/3bb7b8b4/Ederson')
df_list = df_list # Ostatnie 9 DataFrame'ów
keys = list(player_dict.keys())

In [44]:
dict_dfs = {keys[i]: df for i, df in enumerate(df_list)}

len(player_dict['Standard Stats']) == len(dict_dfs['Standard Stats'].columns)

False

In [82]:
len(df_list[3)

Unnamed: 0_level_0           Unnamed: 1_level_0  \
                         Season                          Age   
0                     2012-2013                           18   
1                     2013-2014                           19   
2                     2014-2015                           20   
3                     2015-2016                           21   
4                     2016-2017                           22   
5                     2017-2018                           23   
6                     2018-2019                           24   
7                     2019-2020                           25   
8                     2020-2021                           26   
9                     2021-2022                           27   
10                    2022-2023                           28   
11                    2023-2024                           29   
12                   12 Seasons                   12 Seasons   
13                          NaN                          NaN   
14  Manchester City (7 Seasons)  Manchester City (7 Seasons)   
15          Rio Ave (3 Seasons)          Rio Ave (3 Seasons)   
16          Benfica (2 Seasons)          Benfica (2 Seasons)   
17                          NaN                          NaN   
18   Premier League (7 Seasons)   Premier League (7 Seasons)   
19    Primeira Liga (5 Seasons)    Primeira Liga (5 Seasons)   

             Unnamed: 2_level_0 Unnamed: 3_level_0 Unnamed: 4_level_0  \
                          Squad            Country               Comp   
0                       Rio Ave             pt POR   1. Primeira Liga   
1                       Rio Ave             pt POR   1. Primeira Liga   
2                       Rio Ave             pt POR   1. Primeira Liga   
3                       Benfica             pt POR   1. Primeira Liga   
4                       Benfica             pt POR   1. Primeira Liga   
5               Manchester City            eng ENG  1. Premier League   
6               Manchester City            eng ENG  1. Premier League   
7               Manchester City            eng ENG  1. Premier League   
8               Manchester City            eng ENG  1. Premier League   
9               Manchester City            eng ENG  1. Premier League   
10              Manchester City            eng ENG  1. Premier League   
11              Manchester City            eng ENG  1. Premier League   
12                      3 Clubs                NaN          2 Leagues   
13                          NaN            Country               Comp   
14  Manchester City (7 Seasons)                NaN           1 League   
15          Rio Ave (3 Seasons)                NaN           1 League   
16          Benfica (2 Seasons)                NaN           1 League   
17                          NaN                NaN                NaN   
18   Premier League (7 Seasons)                NaN                NaN   
19    Primeira Liga (5 Seasons)                NaN                NaN   

   Unnamed: 5_level_0 Playing Time                        ... Performance  \
               LgRank           MP  Starts    Min    90s  ...           D   
0                 6th            2       2    180    2.0  ...           0   
1                11th           18      17   1596   17.7  ...           7   
2                10th           17      17   1530   17.0  ...           8   
3                 1st           10      10    890    9.9  ...           0   
4                 1st           27      27   2381   26.5  ...           5   
5                 1st           36      36   3194   35.5  ...           4   
6                 1st           38      38   3420   38.0  ...           2   
7                 2nd           35      35   3071   34.1  ...           3   
8                 1st           36      36   3240   36.0  ...           5   
9                 1st           37      37   3330   37.0  ...           6   
10                1st           35      35   3150   35.0  ...           4

In [65]:
df_list_1=list(df_list[-9].columns)

In [68]:
index = next((i for i, df in enumerate(df_list) if all(col in df.columns for col in df_list_1)), None)

# Wybranie 8 DataFrame'ów po tym, który spełnia kryteria
selected_dfs = df_list[index:index + 9] if index is not None and index + 1 < len(df_list) else []

In [74]:
index_1= next(i for i, df in enumerate(df_list) if len(df.columns)==36)

In [75]:
index_1

4

In [77]:
selected_dfs=df_list[index:index+9]

In [79]:
selected_dfs[0]

Unnamed: 0_level_0           Unnamed: 1_level_0  \
                         Season                          Age   
0                     2021-2022                           16   
1                     2022-2023                           17   
2                     2022-2023                           17   
3                     2023-2024                           18   
4                     2 Seasons                    2 Seasons   
5                           NaN                          NaN   
6   Manchester City (2 Seasons)  Manchester City (2 Seasons)   
7    Manchester City (1 Season)   Manchester City (1 Season)   
8    Manchester City (1 Season)   Manchester City (1 Season)   
9                           NaN                          NaN   
10   Premier League (2 Seasons)   Premier League (2 Seasons)   
11     PL2 — Div. 1 (2 Seasons)     PL2 — Div. 1 (2 Seasons)   

             Unnamed: 2_level_0 Unnamed: 3_level_0 Unnamed: 4_level_0  \
                          Squad            Country               Comp   
0               Manchester City            eng ENG   Jr. PL2 — Div. 1   
1               Manchester City            eng ENG   Jr. PL2 — Div. 1   
2               Manchester City            eng ENG  1. Premier League   
3               Manchester City            eng ENG  1. Premier League   
4                        1 Club                NaN           1 League   
5                           NaN            Country               Comp   
6   Manchester City (2 Seasons)                NaN           1 League   
7    Manchester City (1 Season)                NaN           1 League   
8    Manchester City (1 Season)                NaN           1 League   
9                           NaN                NaN                NaN   
10   Premier League (2 Seasons)                NaN                NaN   
11     PL2 — Div. 1 (2 Seasons)                NaN                NaN   

   Unnamed: 5_level_0 Unnamed: 6_level_0 Playing Time              ...  \
               LgRank                 MP       Starts   Min   90s  ...   
0                 1st                  6            3   312   3.5  ...   
1                 1st                  2            2   180   2.0  ...   
2                 1st                 14           10   903  10.0  ...   
3                 3rd                  9            4   364   4.0  ...   
4                 NaN                 23           14  1267  14.1  ...   
5              LgRank                 MP       Starts   Min   90s  ...   
6                 NaN                 23           14  1267  14.1  ...   
7                 NaN                  6            3   312   3.5  ...   
8                 NaN                  2            2   180   2.0  ...   
9                 NaN                NaN          NaN   NaN   NaN  ...   
10                NaN                 23           14  1267  14.1  ...   
11                NaN                  8            5   492   5.5  ...   

   Per 90 Minutes                                                          \
              Ast   G+A  G-PK  G+A-PK    xG   xAG  xG+xAG  npxG  npxG+xAG   
0            0.00  0.29  0.29    0.29   NaN   NaN     NaN   NaN       NaN   
1            1.00  1.00  0.00    1.00   NaN   NaN     NaN   NaN       NaN   
2            0.00  0.00  0.00    0.00  0.02  0.15    0.17  0.02      0.17   
3            0.00  0.25  0.25    0.25  0.08  0.11    0.19  0.08      0.19   
4            0.00  0.07  0.07    0.07  0.04  0.14    0.17  0.04      0.17   
5             Ast   G+A  G-PK  G+A-PK    xG   xAG  xG+xAG  npxG  npxG+xAG   
6            0.00  0.07  0.07    0.07  0.04  0.14    0.17  0.04      0.17   
7            0.00  0.29  0.29    0.29   NaN   NaN     NaN   NaN       NaN   
8            1.00  1.00  0.00    1.00   NaN   NaN     NaN   NaN       NaN   
9             NaN   NaN   NaN     NaN   NaN   NaN     NaN   NaN       NaN   
10           0.00  0.07  0.07    0.07  0.04  0.14    0.17  0.04      0.17   
11           0.37  0.55  0.18    0.55   NaN   NaN     N